In [13]:
import os
import numpy as np
import pandas as pd

from datetime import datetime
from msal import PublicClientApplication

import holidays
from datetime import date
from datetime import timedelta

from cognite.client import CogniteClient
from cognite.client.data_classes import TimeSeries, Asset

import wapi # wattsights egen pakke / den må installers via `pip install wapi-python`

In [14]:
# Log-in detaljer
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
CDF_CLUSTER = "az-power-no-northeurope"
COGNITE_PROJECT = "heco-dev"

# Code to log-in WIHTOUT client_secret
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000


def authenticate_azure():

    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)

    # interactive login - make sure you have http://localhost:port in Redirect URI in App Registration as type "Mobile and desktop applications"
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    return creds


creds = authenticate_azure()

client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-python-dev",
)

In [15]:
#Definer ønsket kruver
curves = ["NO1_consumption_per_15min",
"NO1_consumption_forecast_per_15min"]

#Definer start dato
start_dato = datetime(2015,1,1)
slutt_dato = datetime(2022,6,3)

df_watt = pd.DataFrame()
for curve in curves:
    print(curve)
    hm = client.datapoints.retrieve_dataframe(
        start=start_dato,
        end=slutt_dato,
        aggregates=["average"],
        granularity="1h",
        id=client.time_series.retrieve(external_id=curve).id,)
    df_watt = pd.merge(df_watt, hm, left_index=True, right_index=True, how="outer")

df_watt

NO1_consumption_per_15min
NO1_consumption_forecast_per_15min


,5172002421720946|average,7859454433446946|average
2015-01-01 00:00:00,4678.090390,4363.471289
2015-01-01 01:00:00,4574.095914,4223.608992
2015-01-01 02:00:00,4498.004329,4157.389157
2015-01-01 03:00:00,4480.165823,4150.039298
2015-01-01 04:00:00,4540.027045,4291.640447
...,...,...
2022-06-02 19:00:00,3062.750000,3317.589204
2022-06-02 20:00:00,3002.375000,3221.822662
2022-06-02 21:00:00,2862.625000,3089.686713
2022-06-02 22:00:00,2684.000000,NaN


In [16]:
df_watt.columns = ["cons_actual","cons_forecast"]

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score

df_watt = df_watt.dropna()

cons_actual = df_watt["cons_actual"].values
cons_forecast = df_watt["cons_forecast"].values

# cons_actual = cons_actual[~np.isnan(cons_actual)]
# cons_forecast = cons_forecast[~np.isnan(cons_forecast)]

print(f"MSE Volue: {mean_squared_error(cons_forecast, cons_actual)}")
print(f"MAE Volue: {mean_absolute_error(cons_forecast, cons_actual)}")
print(f"MSLE Volue: {mean_squared_log_error(cons_forecast, cons_actual)}")
print(f"R2 Volue: {r2_score(cons_forecast, cons_actual)}")

MSE Volue: 33104.503620705815
MAE Volue: 135.34678409485824
MSLE Volue: 0.0019185620902641169
R2 Volue: 0.9806514546231295
